## Архитектуры свёрточных сетей

In [1]:
import torch
from torch import nn
import torchvision as tv
from torchsummary import summary
import time

In [3]:
def evaluate_accuracy(data_iter, net):
    acc_sum, n = 0, 0
    net.eval()
    for X, y in data_iter:
        X, y = X.to(device), y.to(device)
        acc_sum += (net(X).argmax(axis=1) == y).sum()
        n += y.shape[0]
    return acc_sum.item() / n

In [4]:
def train(net, train_iter, test_iter, trainer, num_epochs):
    net.to(device)
    loss = nn.CrossEntropyLoss(reduction='sum')
    net.train()
    for epoch in range(num_epochs):
        train_l_sum, train_acc_sum, n, start = 0.0, 0.0, 0, time.time()
        
        for i, (X, y) in enumerate(train_iter):
            X, y = X.to(device), y.to(device)
            trainer.zero_grad()
            y_hat = net(X)
            l = loss(y_hat, y)
            l.backward()
            trainer.step()
            train_l_sum += l.item()
            train_acc_sum += (y_hat.argmax(axis=1) == y).sum().item()
            n += y.shape[0]

            if i % 10 == 0:
              print(f"Step {i}. time since epoch: {time.time() -  start:.3f}. " 
                    f"Train acc: {train_acc_sum / n:.3f}. Train Loss: {train_l_sum / n:.3f}")
        test_acc = evaluate_accuracy(test_iter, net.to(device))
        print('-' * 20)
        print(f'epoch {epoch + 1}, loss {train_l_sum / n:.4f}, train acc {train_acc_sum / n:.3f}'
              f', test acc {test_acc:.3f}, time {time.time() - start:.1f} sec')

In [5]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [6]:
torch.backends.cuda.max_split_size_mb = 10000

## DataSet

In [7]:
BATCH_SIZE = 192
transoforms = tv.transforms.Compose([
    tv.transforms.Resize((224, 224)),
    tv.transforms.ToTensor()
])
# emnist_train = EMNIST('../data', split='balanced', train=True, download=True, transform=transoforms)
# emnist_test = EMNIST('../data', split='balanced', train=False, download=True, transform=transoforms)

train_dataset = tv.datasets.EMNIST('../data', split='balanced', train=True, download=True, transform=transoforms)
test_dataset = tv.datasets.EMNIST('../data', split='balanced', train=False, download=True, transform=transoforms)
train_iter = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE)
test_iter = torch.utils.data.DataLoader(test_dataset, batch_size=BATCH_SIZE)

## Предобученные архитектуры

In [8]:
transoforms = tv.transforms.Compose([
    tv.transforms.Grayscale(3),
    tv.transforms.Resize((224, 224)),
    tv.transforms.ToTensor()
])

train_dataset = tv.datasets.MNIST('.', train=True, transform=transoforms, download=True)
test_dataset = tv.datasets.MNIST('.', train=False, transform=transoforms, download=True)
train_iter = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE)
test_iter = torch.utils.data.DataLoader(test_dataset, batch_size=BATCH_SIZE)

#### ResNet

In [9]:
model = tv.models.resnet18(pretrained=True)

c:\Users\Анатолий\source\repos\PyTorchtest\PyTorchtest\TorchEnv\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Анатолий\source\repos\PyTorchtest\PyTorchtest\TorchEnv\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [10]:
model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [11]:
summary(model.to(device), input_size=(3, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,408
       BatchNorm2d-2         [-1, 64, 112, 112]             128
              ReLU-3         [-1, 64, 112, 112]               0
         MaxPool2d-4           [-1, 64, 56, 56]               0
            Conv2d-5           [-1, 64, 56, 56]          36,864
       BatchNorm2d-6           [-1, 64, 56, 56]             128
              ReLU-7           [-1, 64, 56, 56]               0
            Conv2d-8           [-1, 64, 56, 56]          36,864
       BatchNorm2d-9           [-1, 64, 56, 56]             128
             ReLU-10           [-1, 64, 56, 56]               0
       BasicBlock-11           [-1, 64, 56, 56]               0
           Conv2d-12           [-1, 64, 56, 56]          36,864
      BatchNorm2d-13           [-1, 64, 56, 56]             128
             ReLU-14           [-1, 64,

In [12]:
# Убираем требование градиента:
for param in model.parameters():
    param.requires_grad = False

In [13]:
model.fc

Linear(in_features=512, out_features=1000, bias=True)

In [14]:
model.fc = nn.Linear(in_features=512, out_features=10)

In [15]:
print("Params to learn:")
params_to_update = []
for name, param in model.named_parameters():
    if param.requires_grad == True:
        params_to_update.append(param)
        print("\t",name)

Params to learn:
	 fc.weight
	 fc.bias


In [16]:
trainer = torch.optim.Adam(params_to_update, lr=0.001)

In [17]:
train(model, train_iter, test_iter, trainer, 1)

Step 0. time since epoch: 0.571. Train acc: 0.104. Train Loss: 2.662
Step 10. time since epoch: 3.750. Train acc: 0.211. Train Loss: 2.220
Step 20. time since epoch: 6.819. Train acc: 0.419. Train Loss: 1.926
Step 30. time since epoch: 9.913. Train acc: 0.537. Train Loss: 1.700
Step 40. time since epoch: 12.882. Train acc: 0.608. Train Loss: 1.526
Step 50. time since epoch: 15.867. Train acc: 0.655. Train Loss: 1.393
Step 60. time since epoch: 18.839. Train acc: 0.692. Train Loss: 1.278
Step 70. time since epoch: 21.816. Train acc: 0.718. Train Loss: 1.190
Step 80. time since epoch: 24.824. Train acc: 0.740. Train Loss: 1.114
Step 90. time since epoch: 27.767. Train acc: 0.757. Train Loss: 1.050
Step 100. time since epoch: 30.853. Train acc: 0.772. Train Loss: 0.992
Step 110. time since epoch: 33.789. Train acc: 0.785. Train Loss: 0.943
Step 120. time since epoch: 36.775. Train acc: 0.796. Train Loss: 0.899
Step 130. time since epoch: 39.728. Train acc: 0.805. Train Loss: 0.862
Step 14

#### vgg11

In [18]:
# VGG11
model_vgg11 = tv.models.vgg11(pretrained=True)


c:\Users\Анатолий\source\repos\PyTorchtest\PyTorchtest\TorchEnv\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG11_Weights.IMAGENET1K_V1`. You can also use `weights=VGG11_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [19]:
model_vgg11

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (11): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (12): ReLU(inplace=True)
    (13): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (14): ReLU(inplace=True)
    (15): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
 

In [20]:
# Убираем требование градиента:
for param in model_vgg11.parameters():
    param.requires_grad = False

In [92]:
model_vgg11.classifier

Sequential(
  (0): Linear(in_features=25088, out_features=4096, bias=True)
  (1): ReLU(inplace=True)
  (2): Dropout(p=0.5, inplace=False)
  (3): Linear(in_features=4096, out_features=4096, bias=True)
  (4): ReLU(inplace=True)
  (5): Dropout(p=0.5, inplace=False)
  (6): Linear(in_features=4096, out_features=1000, bias=True)
)

In [21]:
model_vgg11.classifier = nn.Sequential(
    nn.Linear(25088, 4096), 
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(4096, 10)
)

In [22]:
# Распечатать заголовок секции 
print("Params to learn:")  

# Создать пустой список для хранения обучаемых параметров
params_to_update = []  

# Получить именованные параметры модели в виде списка tuple
for name, param in model_vgg11.named_parameters():
    
    # Проверить флаг requires_grad, который указывает нужно ли обучать этот параметр
    if param.requires_grad == True:

        # Добавить параметр в список обучаемых
        params_to_update.append(param)
        
        # Распечатать имя параметра с отступом
        print("\t",name)
        
# Теперь в params_to_update будут только обучаемые параметры модели

Params to learn:
	 classifier.0.weight
	 classifier.0.bias
	 classifier.3.weight
	 classifier.3.bias


In [23]:
trainer = torch.optim.Adam(params_to_update, lr=0.001)

In [24]:
train(model_vgg11, train_iter, test_iter, trainer, 1)

Step 0. time since epoch: 0.838. Train acc: 0.078. Train Loss: 2.302
Step 10. time since epoch: 5.360. Train acc: 0.753. Train Loss: 0.889
Step 20. time since epoch: 9.733. Train acc: 0.850. Train Loss: 0.539
Step 30. time since epoch: 14.170. Train acc: 0.884. Train Loss: 0.421
Step 40. time since epoch: 18.533. Train acc: 0.905. Train Loss: 0.347
Step 50. time since epoch: 22.917. Train acc: 0.916. Train Loss: 0.304
Step 60. time since epoch: 27.300. Train acc: 0.926. Train Loss: 0.267
Step 70. time since epoch: 31.956. Train acc: 0.933. Train Loss: 0.242
Step 80. time since epoch: 36.386. Train acc: 0.939. Train Loss: 0.223
Step 90. time since epoch: 40.731. Train acc: 0.943. Train Loss: 0.205
Step 100. time since epoch: 45.114. Train acc: 0.947. Train Loss: 0.190
Step 110. time since epoch: 49.597. Train acc: 0.950. Train Loss: 0.178
Step 120. time since epoch: 53.954. Train acc: 0.953. Train Loss: 0.167
Step 130. time since epoch: 58.339. Train acc: 0.955. Train Loss: 0.159
Step 1

### Inception v3 

In [25]:
# Inception v3  
model_inception = tv.models.inception_v3(pretrained=True)


c:\Users\Анатолий\source\repos\PyTorchtest\PyTorchtest\TorchEnv\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=Inception_V3_Weights.IMAGENET1K_V1`. You can also use `weights=Inception_V3_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [26]:

# Создайте класс для модифицированной версии Inception v3 с ядром (3 x 3)
class ModifiedInceptionV3(nn.Module):
    def __init__(self):
        super(ModifiedInceptionV3, self).__init__()
        self.inception = tv.models.inception_v3(pretrained=True)  # Загрузите предварительно обученную Inception v3

        # Замените сверточные слои с ядром (5 x 5) на слои с ядром (3 x 3)
        for name, module in self.inception.named_children():
            if isinstance(module, nn.Conv2d) and module.kernel_size == (5, 5):
                setattr(self.inception, name, nn.Conv2d(
                    module.in_channels, module.out_channels, kernel_size=(3, 3), stride=module.stride, padding=module.padding))

    def forward(self, x):
        return self.inception(x)

# Создайте экземпляр модифицированной модели
model_inception_modified = ModifiedInceptionV3()

# Выведите информацию о модели
summary(model_inception_modified.to(device), input_size=(3, 299, 299))  # Указываем размер входного изображения 299 x 299

# Выведите информацию о модели
print(model_inception_modified)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 149, 149]             864
       BatchNorm2d-2         [-1, 32, 149, 149]              64
       BasicConv2d-3         [-1, 32, 149, 149]               0
            Conv2d-4         [-1, 32, 147, 147]           9,216
       BatchNorm2d-5         [-1, 32, 147, 147]              64
       BasicConv2d-6         [-1, 32, 147, 147]               0
            Conv2d-7         [-1, 64, 147, 147]          18,432
       BatchNorm2d-8         [-1, 64, 147, 147]             128
       BasicConv2d-9         [-1, 64, 147, 147]               0
        MaxPool2d-10           [-1, 64, 73, 73]               0
           Conv2d-11           [-1, 80, 73, 73]           5,120
      BatchNorm2d-12           [-1, 80, 73, 73]             160
      BasicConv2d-13           [-1, 80, 73, 73]               0
           Conv2d-14          [-1, 192,

In [27]:

# Убираем требование градиента
for param in model_inception_modified.parameters():
    param.requires_grad = False
    


In [28]:
model_inception_modified

ModifiedInceptionV3(
  (inception): Inception3(
    (Conv2d_1a_3x3): BasicConv2d(
      (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
      (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    )
    (Conv2d_2a_3x3): BasicConv2d(
      (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    )
    (Conv2d_2b_3x3): BasicConv2d(
      (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    )
    (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (Conv2d_3b_1x1): BasicConv2d(
      (conv): Conv2d(64, 80, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(80, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    )
    (Conv2d_4a

In [29]:

model_inception_modified.fc = nn.Linear(2048, 10)


In [30]:
print("Params to learn:")
params_to_update = []
for name, param in model_inception_modified.named_parameters():
    if param.requires_grad == True:
        params_to_update.append(param)
        print("\t",name)

Params to learn:
	 fc.weight
	 fc.bias


In [31]:
trainer = torch.optim.Adam(params_to_update, lr=0.001) 

In [32]:

train(model_inception, train_iter, test_iter, trainer, 1)

RuntimeError: Calculated padded input size per channel: (3 x 3). Kernel size: (5 x 5). Kernel size can't be greater than actual input size